# Практическое задание к уроку 2. Feed-forward neural network

Итак, давайте потренируемся тренировать нейронные сети прямого распространения (так как делали на паре)
При этом попробуем создать свою функцию активации на одном из слоев

In [6]:
# Сделаем необходимые импорты
import torch
import numpy as np

from torch import nn
from torch import optim
import torchvision
import torchvision.transforms as transforms
from tqdm import tqdm
import torch.nn.functional as F

In [7]:
# Загрузим датасет CIFAR-100, сразу же создадим dataloader для него
# Если вам не хватает вычислительных ресурсов, то можно вернуться к CIFAR-10
train_dataset = torchvision.datasets.CIFAR100(root='data/',
                                             train=True,  
                                             transform=transforms.ToTensor(), 
                                             download=True)

Files already downloaded and verified


In [8]:
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=64, 
                                           shuffle=True)

In [9]:
image, label = train_dataset[0]
print (image.size())
print (label)

torch.Size([3, 32, 32])
19


In [10]:
# Создайте собственную архитектуру! Можете использовать все, что угодно, но в ограничении на использование линейные слои (пока без сверток)
# Давайте добавим ограниченный Leaky_relu, то есть output = max(0.1x, 0.5x)
# Ваша задача добавить его в архитектуру сети как функцию активации
class SimpleNet(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super().__init__()
        self.fc1 = nn.Linear(input_dim, 4 * hidden_dim)
        self.b1 = nn.BatchNorm1d(4 * hidden_dim)
        self.fc2 = nn.Linear(4 * hidden_dim,  2 * hidden_dim)
        self.b2 = nn.BatchNorm1d(2 * hidden_dim)
        self.fc3 = nn.Linear(2 * hidden_dim, hidden_dim)
        self.b3 = nn.BatchNorm1d(hidden_dim)
        self.fc4 = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        x = x.view(x.shape[0], -1)
        x = F.leaky_relu(self.fc1(x), 0.05)
        x = self.b1(x)
        x = F.leaky_relu(self.fc2(x), 0.05)
        x = self.b2(x)
        x = F.leaky_relu(self.fc3(x), 0.05)
        x = self.b3(x)
        x = torch.sigmoid(self.fc4(x))
        return x


net = SimpleNet(3072, 1000, 100)

In [11]:
# Запустить обучение (по аналогии с тем, что делали на паре)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), lr=0.001)

In [12]:
for epoch in tqdm(range(10)):  
    running_loss = 0.0
    for i, data in enumerate(train_loader, 0):
        inputs, labels = data[0], data[1]

        # обнуляем градиент
        optimizer.zero_grad()

        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # выводим статистику о процессе обучения
        running_loss += loss.item()
        if i % 300 == 0:    # печатаем каждые 300 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 2000))
            running_loss = 0.0

print('Training is finished!')

  0%|                                                    | 0/10 [00:00<?, ?it/s]

[1,     1] loss: 0.002
[1,   301] loss: 0.653
[1,   601] loss: 0.640


 10%|████▎                                      | 1/10 [02:25<21:50, 145.56s/it]

[2,     1] loss: 0.002
[2,   301] loss: 0.630
[2,   601] loss: 0.628


 20%|████████▌                                  | 2/10 [04:50<19:22, 145.37s/it]

...

[9,     1] loss: 0.002
[9,   301] loss: 0.612
[9,   601] loss: 0.613


 90%|██████████████████████████████████████▋    | 9/10 [22:23<02:28, 149.00s/it]

[10,     1] loss: 0.002
[10,   301] loss: 0.611
[10,   601] loss: 0.611


100%|██████████████████████████████████████████| 10/10 [24:49<00:00, 148.99s/it]

Training is finished!
